ChainerでCNNしたった
https://qiita.com/yoyoyo_/items/14f243c058f5a1e7044b

In [1]:
import chainer
from chainer.dataset import convert
import chainer.links as L
import chainer.functions as F
from chainer import serializers
from chainer.links.caffe import CaffeFunction

import glob, random, os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time
import _pickle as pickle

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## fine-tuning setting
FT_MODEL = 'result/model.npz'

## paths to train image directory and test image one
HOME = os.path.expanduser('~')
TRAIN_PATH = HOME + '/Deeplearning/Images2/Train/'
TEST_PATH = HOME + '/Deeplearning/Images2/Test/'

## model name for save trained, and model name for load testing
SAVE_MODEL = 'result/model2'
LOAD_MODEL = 'result/model2.npz'

## Train hyper-parameters
CLASS = 2
INPUT_WIDTH = 128
INPUT_HEIGHT = 128
MINIBATCH_SIZE = 20
LEARN_RATE = 0.001
EPOCH = 50
GPU = -1

In [ ]:
class Mynet(chainer.Chain):

    def __init__(self, class_labels=10):
        super(Mynet, self).__init__()
        with self.init_scope():
            self.conv1_1 = L.Convolution2D(None, 16, ksize=5, pad=2, nobias=True)
            self.conv1_2 = L.Convolution2D(None, 16, ksize=5, pad=2, nobias=True)
            self.conv2_1 = L.Convolution2D(None, 32, ksize=3, pad=1, nobias=True)
            self.conv2_2 = L.Convolution2D(None, 32, ksize=3, pad=1, nobias=True)
            self.fc1 = L.Linear(None, 512, nobias=True)
            self.fc2 = L.Linear(None, class_labels, nobias=True)      

    def __call__(self, x):
        conv1_1 = self.conv1_1(x)
        conv1_1 = F.relu(conv1_1)
        conv1_2 = self.conv1_2(conv1_1)
        conv1_2 = F.relu(conv1_2)
        pool1 = F.max_pooling_2d(conv1_2, ksize=2, stride=2)
        conv2_1 = self.conv2_1(pool1)
        conv2_1 = F.relu(conv2_1)
        conv2_2 = self.conv2_2(conv2_1)
        conv2_2 = F.relu(conv2_2)
        pool2 = F.max_pooling_2d(conv2_2, ksize=2, stride=2)
        fc1 = self.fc1(pool2)
        fc1 = F.relu(fc1)
        fc2 = self.fc2(fc1)
        return fc2

In [ ]:
INPUT_WIDTH = 128
INPUT_HEIGHT = 128

def load_images(dataset_path, shuffle=True):
    filepaths_jpg = glob.glob(dataset_path + '/*.jp*g')
    filepaths_png = glob.glob(dataset_path + '/*.png')
    filepaths = filepaths_jpg + filepaths_png
    filepaths.sort()
    datasets = []
    for filepath in filepaths:
        img = Image.open(filepath).convert('RGB') ## Gray->L, RGB->RGB
        img = img.resize((INPUT_WIDTH, INPUT_HEIGHT))

        x = np.array(img, dtype=np.float32)
        ## Normalize [0, 255] -> [0, 1]
        x = x / 255.
        ## Reshape image to input shape of CNN
        x = x.transpose(2, 0, 1)
        #x = x.reshape(3, INPUT_HEIGHT, INPUT_WIDTH)

        ## Get label(ground-truth) from file name path 
        label = int(filepath.split('/')[-1].split('_')[0])
        t = np.array(label, dtype=np.int32)
        datasets.append((x,t))

    if shuffle: random.shuffle(datasets)

    return datasets, filepaths

In [ ]:
def main_train(train_Model, ft_Model=None):

    print('\nmodel training start!!\n')
    print('# GPU: {}'.format(GPU))
    print('# Minibatch-size: {}'.format(MINIBATCH_SIZE))
    print('# Epoch: {}'.format(EPOCH))
    print('# Learnrate: {}'.format(LEARN_RATE))

    ## Load train and test images 
    train, _ = load_images(TRAIN_PATH)
    test, _ = load_images(TEST_PATH)

    if len(train) < 1 or len(test) < 1:
        raise Exception('train num : {}, test num: {}'.format(len(train), len(test)))

    train_count = len(train)
    test_count = len(test)

    print('# Train images: {}'.format(train_count))
    print('# Test images: {}\n'.format(test_count))

    ## model define
    model = train_Model

    ## Set GPU device
    if GPU >= 0:
        chainer.cuda.get_device(GPU).use()
        model.to_gpu()

    ## Fine-tuning from pre-trained model
    if ft_Model is not None:
        #orig = pickle.load(open(FT_MODEL, "rb"))

        ## train model parameter initialization
        dummy = np.zeros((3, INPUT_HEIGHT, INPUT_WIDTH))
        dummy = dummy[None, ...]
        dummy = np.array(dummy, dtype=np.float32)
        dummy = chainer.Variable(dummy)
        _, _ = model(dummy)

        ## load fine-tuning original model and copy it to train model
        print('\nFine-tuning from {}\n'.format(FT_MODEL))
        orig = ft_Model
        serializers.load_npz(FT_MODEL, orig)
        fine_tuning(orig, model)

        #serializers.load_npz(FT_MODEL, model)


    ## Set Optimizer
    optimizer = chainer.optimizers.MomentumSGD(LEARN_RATE)
    optimizer.setup(model)
    optimizer.add_hook(chainer.optimizer.WeightDecay(5e-4))

    train_iter = chainer.iterators.SerialIterator(train, MINIBATCH_SIZE)
    test_iter = chainer.iterators.SerialIterator(test, MINIBATCH_SIZE, repeat=False, shuffle=False)

    ## Training start!!
    start = time.time()

    print('epoch  train_loss  train_accuracy  test_loss  test_accuracy  Elapsed-Time')

    while train_iter.epoch < EPOCH:

        batch = train_iter.next()
        # Reduce learning rate by 0.5 every 25 epochs.
        #if train_iter.epoch % 25 == 0 and train_iter.is_new_epoch:
        #    optimizer.lr *= 0.5
        #    print('Reducing learning rate to: ', optimizer.lr)

        train_losses = []
        train_accuracies = []

        x_array, t_array = convert.concat_examples(batch, GPU)
        x = chainer.Variable(x_array)
        t = chainer.Variable(t_array)

        y, _ = model(x)
        loss_train = F.softmax_cross_entropy(y, t)
        accuracy_train = F.accuracy(y, t)
        model.cleargrads()
        loss_train.backward()
        optimizer.update()

        train_losses.append(chainer.cuda.to_cpu(loss_train.data))
        accuracy_train.to_cpu()
        train_accuracies.append(accuracy_train.data)

        if train_iter.is_new_epoch:
            #print('epoch: ', train_iter.epoch)
            #print('train mean loss: {:.2f}, accuracy: {:.2f}'.format( sum_loss_train / train_count, sum_accuracy_train / train_count))
            # evaluation

            test_losses = []
            test_accuracies = []

            sum_accuracy_test = 0
            sum_loss_test = 0
            #model.predictor.train = False
            for batch in test_iter:
                x_array, t_array = convert.concat_examples(batch, GPU)
                x = chainer.Variable(x_array)
                t = chainer.Variable(t_array)

                y = model(x)

                loss_test = F.softmax_cross_entropy(y, t)
                accuracy_test = F.accuracy(y, t)

                test_losses.append(chainer.cuda.to_cpu(loss_test.data))
                accuracy_test.to_cpu()
                test_accuracies.append(accuracy_test.data)


            test_iter.reset()
            #model.predictor.train = True

            print('{:>5}  {:^10.4f}  {:^14.4f}  {:^9.4f}  {:^13.4f}  {:^12.2f}'.format(train_iter.epoch, np.mean(train_losses), np.mean(train_accuracies), np.mean(test_losses), np.mean(test_accuracies), time.time()-start))


    print('\ntraining finished!!\n')

    ## Save the model and the optimizer
    print('save model start!!\n')
    directory = SAVE_MODEL.split('/')[0]
    if not os.path.exists(directory):
        os.system('mkdir {}'.format(directory))
        print('make outout model directory --> {}'.format(directory))

    serializers.save_npz(SAVE_MODEL + '.npz', model)
    print('save the model --> {}'.format(SAVE_MODEL + '.npz') )
    serializers.save_npz(SAVE_MODEL + '.state', optimizer)
    print('save the optimizer --> {}'.format(SAVE_MODEL + '.state'))
    pickle.dump(model, open(SAVE_MODEL + '.pkl', 'wb'))
    print('save the model --> {}'.format(SAVE_MODEL + '.pkl'))

    print('\nmodel save finished!!\n')